### Mini Project - Object Tracking

OpenCV 2.7.X only

In [8]:
import cv2
import numpy as np


# defines the color purple range in HSV
lower_purple = np.array([130, 50, 90])
upper_purple = np.array([170, 255, 255])

# creates an empty array that stores our points
points = []

kernel = (3, 3)
frame_count = 0

In [12]:
# code that initializes the webcam
cap = cv2.VideoCapture(0)

# get default cam window size
ret, frame = cap.read()
Height, Width = frame.shape[:2]



while True:
    
    ret, frame = cap.read()
    hsv_img = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # threshold the hsv images to track only object in  purle colors
    mask = cv2.inRange(hsv_img, lower_purple, upper_purple)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)     
    contours, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # creates an empty array to store the centroid of mass
    center = int(Height/2), int(Width/2)
    
    # countours is found
    if len(contours) > 0:
        
        # gets the largest contour using area and its center
        c = max(contours, key=cv2.contourArea)                 
        (x, y), radius = cv2.minEnclosingCircle(c)
        M = cv2.moments(c)
        try:
            center = (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))
        except:
            center = int(Height/2), int(Width/2)
        
        # Takes only contours that have larger than 18 pixel radius
        if radius > 18:
            # Draw Circle and leace the last center creating a trail
            cv2.circle(frame, (int(x), int(y)), int(radius), (0,0,255), 2)
            cv2.circle(frame, center, 5, (0, 255, 0), -1)
            
        # logs the center points
        points.append(center)
            
        # Code that loops over the set of tracked points with radius pixel greater than 18
        if radius > 18:
            for i in range(1, len(points)):
                try:
                    cv2.line(frame, points[i - 1], points[i], (0, 255, 0), 2)
                except:
                    pass
            frame_count = 0
                
    else:
        frame_count += 1
                
        # clears trails everytime our object has been out for 10 frames
        if frame_count == 10:
            points = []
            frame_count = 0
                    
                    
    # cturns image into a mirror like view
    frame = cv2.flip(frame, 1)
    cv2.imshow('Object OR Ball Tracker', frame)
    
    cv2.imshow("mask", mask)
            
    if cv2.waitKey(1) == 13:
        break
                
cap.release()
cv2.destroyAllWindows()
                    